# Bài thực hành số 3.1

Chào mừng đến với bài thực hành số 3.1, trong bài thực hành này ta sẽ học cách sử dụng thuật toán DeepSORT với model YOLOv5 để tracking vật thể trong video. 

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể hiểu được quá trình áp dụng DeepSORT và YOLO lên video.


### Cài đặt thư viện cần thiết

In [21]:
import numpy as np
import torch
import cv2
import math
import time
from PIL import Image
from deep_sort_realtime.deepsort_tracker import DeepSort
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Tải model nhận diện

Đáp án bài tập 1

In [ ]:
# force reload: tránh xung đột tham số khi load model mới
model = torch.hub.load('ultralytics/yolov5', 'custom', path= r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\custom_yolov5_car_v2\weights\best.pt", force_reload = True)

### Khởi tạo DeepSORT

Đáp án bài tập 2

In [4]:
# Khởi tạo DeepSORT
object_tracker = DeepSort(max_age=3,
                          n_init=2,
                          nms_max_overlap=1.0,
                          max_cosine_distance=0.3)

### Các hàm xử lý đầu vào cho thuật toán DeepSORT

Đáp án bài tập 3

In [23]:
def score_frame(frame):
	results = model(frame)
	labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
	return labels, cord

classes = model.names

def class_to_label(x):
	return classes[int(x)]

def plot_boxes(results, height, width, confidence=0.3):
	labels, cord = results
	detections = []

	for i in range(len(labels)):
		row = cord[i]
		if row[4] >= confidence:
			x1, y1, x2, y2 = int(
				row[0]*width), int(row[1]*height), int(row[2]*width), int(row[3]*height)

			conf = float(row[4].item())
			class_label = class_to_label(labels[i])
			# print(feature)
			detections.append(
				([x1, y1, int(x2-x1), int(y2-y1)], conf, class_label))

	return detections

### Áp dụng DeepSORT và YOLO vào video

Đáp án bài tập 4

In [27]:
import cv2

input_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2.mp4"  # Đường dẫn đến video đầu vào
output_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2_output_SORT.mp4"  # Đường dẫn đến video đầu ra

# Mở video đầu vào
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Lấy thông số của video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho định dạng MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
while cap.isOpened():
	ret, frame = cap.read()
	if not ret:
		break

	# Dự đoán các đối tượng trong khung hình

	results = score_frame(frame)
	detections = plot_boxes(results=results, height=frame.shape[0], width=frame.shape[1], confidence=0.5)
	tracks = object_tracker.update_tracks(detections, frame=frame)

	# Vẽ các khung hình và các ID theo dõi lên video
	for track in tracks:
		bbox = track.to_tlbr()  # Bounding box dưới dạng (x1, y1, x2, y2)
		track_id = track.track_id  # ID theo dõi
		x1, y1, x2, y2 = map(int, bbox)
		cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
		cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


	# Ghi khung hình vào video đầu ra
	out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
